In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
tf.config.list_physical_devices()

2021-11-25 23:25:09.246979: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-25 23:25:09.351410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 23:25:09.351682: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2070 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2021-11-25 23:25:09.352599: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-25 23:25:09.369699: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-25 23:25:09.379598: I tensorflow/stream_executor/platform/de

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

mmon_runtime/gpu/gpu_device.cc:1703] Adding visible gpu devices: 0


In [3]:
# Downloading the data
!wget https://www.dropbox.com/s/tp3l54tnatvbldf/bbc.csv?dl=0 -O 'bbc.csv'

--2021-11-25 23:25:09--  https://www.dropbox.com/s/tp3l54tnatvbldf/bbc.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tp3l54tnatvbldf/bbc.csv [following]
--2021-11-25 23:25:10--  https://www.dropbox.com/s/raw/tp3l54tnatvbldf/bbc.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5fd1d3b4c4f0bcf93d71aabd9d.dl.dropboxusercontent.com/cd/0/inline/BapUc09RnXe7iSy2UpIB5K2Qo7qju277O4-5jDk4kLao22FS1UBD2rtDtgkvjFAOc_w6zdlDrfJ8JQTzX3bbxEneeZxeCDAfTfmCEa5dcXagRc1hlZwJtArtY12x261eDKRH6VQbbYWqff0ASP0kQWqP/file# [following]
--2021-11-25 23:25:10--  https://uc5fd1d3b4c4f0bcf93d71aabd9d.dl.dropboxusercontent.com/cd/0/inline/BapUc09RnXe7iSy2UpIB5K2Qo7qju277O4-5jDk4kLao22FS1UBD2rtDtgkvjFAOc_w6zdlDrfJ8JQTzX3bbxEneeZxeCDAfT

In [4]:
# All general imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split 

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Reshape, Conv2D, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Bidirectional, GlobalAveragePooling1D, GRU, GlobalMaxPooling1D, concatenate
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM, GRU, Conv1D, MaxPool1D, Activation, Add

from keras.models import Model, Sequential
from keras.layers.core import SpatialDropout1D

from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, Softmax
from keras.layers import SimpleRNN, GRU, LSTM, BatchNormalization
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import backend as K

from keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import io, os, gc

Using TensorFlow backend.


In [5]:
# loading data
df = pd.read_csv("bbc.csv")
print(df.columns)
le = LabelEncoder()
input_labels = le.fit_transform(df['Class'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
print(df.head())

Index(['Unnamed: 0', 'Article', 'Class'], dtype='object')
{'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}
   Unnamed: 0                                            Article     Class
0           0  Ad sales boost Time Warner profit\n\nQuarterly...  business
1           1  Dollar gains on Greenspan speech\n\nThe dollar...  business
2           2  Yukos unit buyer faces loan claim\n\nThe owner...  business
3           3  High fuel prices hit BA's profits\n\nBritish A...  business
4           4  Pernod takeover talk lifts Domecq\n\nShares in...  business


In [6]:
# Pre-processing data
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
import re
porter_stemmer = PorterStemmer()
doc_list = df['Article'].tolist()
processed_list = []
for doc in doc_list:
  doc_str = re.sub(r'[^\w\s]', '', remove_stopwords(doc))
  doc_str_stem_lst = [porter_stemmer.stem(word) for word in doc_str.split(" ")]
  doc_str_stem = " ".join(doc_str_stem_lst)
  processed_list.append(doc_str_stem)

In [7]:
# Defining the tokenizer
def get_tokenizer(data):
  print('Training tokenizer...')
  tokenizer = Tokenizer()
  print('Read {} Sentences'.format(len(data)))
  tokenizer.fit_on_texts(data)
  return tokenizer

In [8]:
def get_data(tokenizer, MAX_LENGTH, input_data, input_labels):
  print('Loading data')
  
  assert len(input_data) == len(input_labels)
  sequences = tokenizer.texts_to_sequences(input_data)
  X = pad_sequences(sequences, maxlen=MAX_LENGTH)
  Y_bcc = np.array(input_labels)

  return X, Y_bcc

In [9]:
tokenizer = get_tokenizer(processed_list)

Training tokenizer...
Read 1912 Sentences


In [10]:
MAX_LENGTH = 50
# read ml data
X, Y_bcc = get_data(tokenizer, MAX_LENGTH, processed_list, input_labels)

Loading data


In [11]:
# Creating one-hot encodings
y_bcc_labels = keras.utils.np_utils.to_categorical(Y_bcc)
print(y_bcc_labels)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [12]:
# Splitting data into train, val and test
train1_X, test_X, train1_Y, test_Y = train_test_split(X, y_bcc_labels, test_size=0.2, random_state=43)
train_X, val_X, train_Y, val_Y = train_test_split(train1_X, train1_Y, test_size=0.1, random_state=43)

In [13]:
# Setting hyper-parameters
NUM_CLASSES = 5

MAX_SEQUENCE_LENGTH = MAX_LENGTH

MAX_NUM_WORDS = len(tokenizer.word_index) + 1

NUM_EMBEDDING_DIM = 100

In [14]:
# Defining the model
print('Getting Text FF NN')
input_layer = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
embedding_layer = Embedding(MAX_NUM_WORDS, NUM_EMBEDDING_DIM, trainable=True)
embedded_input = embedding_layer(input_layer)
flatten_input = Flatten()(embedded_input)
dense_layer_1 = Dense(2048, activation='tanh')
drop_1 = Dropout(0.1)
dense_layer_2 = Dense(1024, activation='tanh')
drop_2 = Dropout(0.1)
dense_layer_3 = Dense(512, activation='tanh')
drop_3 = Dropout(0.1)
dense_layer_4 = Dense(5, activation='softmax')
output_layer = dense_layer_4(drop_3(dense_layer_3(drop_2(dense_layer_2(drop_1(dense_layer_1(flatten_input)))))))
FFN_model = Model(
    inputs=input_layer, 
    outputs=output_layer)
print(FFN_model.summary())

Getting Text FF NN


2021-11-25 23:25:16.620582: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-11-25 23:25:16.647419: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3000000000 Hz
2021-11-25 23:25:16.647913: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ef50ad48b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-25 23:25:16.647943: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-25 23:25:16.648671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 23:25:16.648982: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeF

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 100)           2253200   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              10242048  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0   

In [15]:
# Defining the model
print('Getting Text RNN')
input_layer = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
embedding_layer = Embedding(MAX_NUM_WORDS, NUM_EMBEDDING_DIM, trainable=True)
embedded_input = embedding_layer(input_layer)
rnn_layer_1 = SimpleRNN(256, activation='relu', return_sequences = False)
drop_1 = Dropout(0.1)
rnn_layer_2 = SimpleRNN(512, activation='relu', return_sequences = False)
drop_2 = Dropout(0.1)
dense_layer_1 = Dense(128, activation='tanh')
dense_layer_2 = Dense(5, activation='softmax')
output_1 = drop_1(rnn_layer_1(embedded_input))
output_2 = drop_2(rnn_layer_2(embedded_input))
output_layer = dense_layer_2(dense_layer_1(Concatenate(axis=1)([output_1, output_2])))
RNN_model = Model(
    inputs=input_layer, 
    outputs=output_layer)
print(RNN_model.summary())

Getting Text RNN
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 50, 100)      2253200     input_2[0][0]                    
__________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)        (None, 256)          91392       embedding_2[0][0]                
__________________________________________________________________________________________________
simple_rnn_2 (SimpleRNN)        (None, 512)          313856      embedding_2[0][0]                
___________________________________________________________________________

In [16]:
from tensorflow.keras.optimizers import Adam
lr = 1e-3
opt = Adam(learning_rate=lr, decay=lr/50)
RNN_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

FFN_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [17]:
BATCH_SIZE = 512
NUM_EPOCHS = 100
# stop = [EarlyStopping(monitor='val_loss', patience=5)]
history = RNN_model.fit(x=train_X,
                    y=train_Y,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(
                      val_X, 
                      val_Y
                    ),
                    shuffle=True,
                    # callbacks=stop,
          )

history = FFN_model.fit(x=train_X,
                    y=train_Y,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(
                      val_X, 
                      val_Y
                    ),
                    shuffle=True,
                    # callbacks=stop,
          )

# Saving the FF NN Converged Model
FFN_model.save("converged_ff_nn_1")
pre_FFN_model = keras.models.load_model("converged_ff_nn_1")
history = pre_FFN_model.fit(x=train_X,
                    y=train_Y,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(
                      val_X, 
                      val_Y
                    ),
                    shuffle=True,
                    # callbacks=stop,
          )

/home/sriram/anaconda3/envs/btp/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Train on 1376 samples, validate on 153 samples
Epoch 1/100


2021-11-25 23:25:19.531659: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


1376/1376 [==============================] - 1s 507us/step - loss: 1.5888 - accuracy: 0.2449 - val_loss: 1.9337 - val_accuracy: 0.2810
Epoch 2/100
1376/1376 [==============================] - 0s 74us/step - loss: 1.6689 - accuracy: 0.2849 - val_loss: 1.5685 - val_accuracy: 0.2941
Epoch 3/100
1376/1376 [==============================] - 0s 72us/step - loss: 1.5287 - accuracy: 0.4840 - val_loss: 1.5674 - val_accuracy: 0.2614
Epoch 4/100
1376/1376 [==============================] - 0s 67us/step - loss: 1.5013 - accuracy: 0.6025 - val_loss: 1.5535 - val_accuracy: 0.3072
Epoch 5/100
1376/1376 [==============================] - 0s 68us/step - loss: 1.4476 - accuracy: 0.6381 - val_loss: 1.5349 - val_accuracy: 0.2810
Epoch 6/100
1376/1376 [==============================] - 0s 70us/step - loss: 1.3610 - accuracy: 0.5036 - val_loss: 1.5158 - val_accuracy: 0.3007
Epoch 7/100
1376/1376 [==============================] - 0s 82us/step - loss: 1.2347 - accuracy: 0.6126 - val_loss: 1.4514 - val_accura

In [18]:
# Getting predictions
from sklearn import metrics
from sklearn.metrics import classification_report
predictions_FFN = FFN_model.predict(test_X)
predictions_pre_FFN = pre_FFN_model.predict(test_X)
predictions_RNN = RNN_model.predict(test_X)

In [19]:
# FF NN - 1 Predictions
predictions = FFN_model.predict(test_X)
y_pred_initial_ff_nn = [idx for idx in np.argmax(predictions, axis=1)]
y_true = [idx for idx in np.argmax(test_Y, axis=1)]
print('FF NN - 1BBC News Classification Accuracy is')
print(metrics.accuracy_score(y_true, y_pred_initial_ff_nn)*100)
print(classification_report(y_true, y_pred_initial_ff_nn, target_names = list(le_name_mapping.keys())))

FF NN - 1BBC News Classification Accuracy is
69.45169712793734
               precision    recall  f1-score   support

     business       0.84      0.62      0.72        79
entertainment       0.66      0.65      0.66        69
     politics       0.63      0.86      0.73        93
        sport       0.72      0.70      0.71        47
         tech       0.69      0.62      0.66        95

     accuracy                           0.69       383
    macro avg       0.71      0.69      0.69       383
 weighted avg       0.71      0.69      0.69       383



In [20]:
# FF NN - 2 Predictions
predictions = pre_FFN_model.predict(test_X)
y_pred_pre_trained_ff_nn = [idx for idx in np.argmax(predictions, axis=1)]
print('FF NN - 2 BBC News Classification Accuracy is')
print(metrics.accuracy_score(y_true, y_pred_pre_trained_ff_nn)*100)
print(classification_report(y_true, y_pred_pre_trained_ff_nn, target_names = list(le_name_mapping.keys())))

FF NN - 2 BBC News Classification Accuracy is
69.19060052219321
               precision    recall  f1-score   support

     business       0.84      0.62      0.72        79
entertainment       0.66      0.65      0.66        69
     politics       0.63      0.86      0.73        93
        sport       0.71      0.68      0.70        47
         tech       0.69      0.62      0.66        95

     accuracy                           0.69       383
    macro avg       0.71      0.69      0.69       383
 weighted avg       0.71      0.69      0.69       383



In [21]:
# RNN Predictions
predictions = RNN_model.predict(test_X)
y_pred_rnn = [idx for idx in np.argmax(predictions, axis=1)]
print('RNN BBC News Classification Accuracy is')
print(metrics.accuracy_score(y_true, y_pred_rnn)*100)
print(classification_report(y_true, y_pred_rnn, target_names = list(le_name_mapping.keys())))

RNN BBC News Classification Accuracy is
77.02349869451697
               precision    recall  f1-score   support

     business       0.76      0.87      0.81        79
entertainment       0.75      0.80      0.77        69
     politics       0.77      0.80      0.78        93
        sport       0.84      0.79      0.81        47
         tech       0.76      0.63      0.69        95

     accuracy                           0.77       383
    macro avg       0.78      0.78      0.77       383
 weighted avg       0.77      0.77      0.77       383



In [22]:
# Saving output predictions in csv file
init_df = pd.DataFrame()
init_df['predictions'] = y_pred_initial_ff_nn
init_df.to_csv('initial_ff_nn.csv')

pre_train_df = pd.DataFrame()
pre_train_df['predictions'] = y_pred_pre_trained_ff_nn
pre_train_df.to_csv('pre_trained_ff_nn.csv')

rnn_df = pd.DataFrame()
rnn_df['predictions'] = y_pred_rnn
rnn_df.to_csv('rnn.csv')

In [23]:
# Implementing Max Voting

max_vote_predictions = []
for i in range(len(y_pred_rnn)):
  class_counts = {0:0, 1:0, 2:0, 3:0, 4:0}
  class_counts[y_pred_initial_ff_nn[i]] += 1
  class_counts[y_pred_pre_trained_ff_nn[i]] += 1
  class_counts[y_pred_rnn[i]] += 1
  sorted_dict = dict(sorted(class_counts.items(), key=lambda item: item[1], reverse=True))
  sorted_classes = list(sorted_dict.keys())
  most_frequent = sorted_classes[0]
  max_vote_predictions.append(most_frequent)

In [24]:
# Max Vote Results
print('MAX VOTE Ensemble BBC News Classification Accuracy is')
print(metrics.accuracy_score(y_true, max_vote_predictions)*100)
print(classification_report(y_true, max_vote_predictions, target_names = list(le_name_mapping.keys())))

MAX VOTE Ensemble BBC News Classification Accuracy is
69.45169712793734
               precision    recall  f1-score   support

     business       0.84      0.62      0.72        79
entertainment       0.66      0.65      0.66        69
     politics       0.63      0.86      0.73        93
        sport       0.72      0.70      0.71        47
         tech       0.69      0.62      0.66        95

     accuracy                           0.69       383
    macro avg       0.71      0.69      0.69       383
 weighted avg       0.71      0.69      0.69       383



In [27]:
# Weighted Voting

# Getting weights
predictions_1 = FFN_model.predict(val_X)
pred_1 = [idx for idx in np.argmax(predictions_1, axis=1)]
predictions_2 = pre_FFN_model.predict(val_X)
pred_2 = [idx for idx in np.argmax(predictions_2, axis=1)]
predictions_3 = RNN_model.predict(val_X)
pred_3 = [idx for idx in np.argmax(predictions_3, axis=1)]
y_val_true = [idx for idx in np.argmax(val_Y, axis=1)]
wt_initial_ff_nn = metrics.accuracy_score(y_val_true, pred_1)
wt_pre_trained_ff_nn = metrics.accuracy_score(y_val_true, pred_2)
wt_rnn = metrics.accuracy_score(y_val_true, pred_3)
print('FF NN 1 weight is', wt_initial_ff_nn)
print('FF NN 2 weight is', wt_pre_trained_ff_nn)
print('RNN weight is', wt_rnn)

FF NN 1 weight is 0.6666666666666666
FF NN 2 weight is 0.6666666666666666
RNN weight is 0.803921568627451


In [28]:
# Implement Weighted Voting
weight_vote_predictions = []
for i in range(len(y_pred_rnn)):
  class_counts = {0:0, 1:0, 2:0, 3:0, 4:0}
  class_counts[y_pred_initial_ff_nn[i]] += wt_initial_ff_nn
  class_counts[y_pred_pre_trained_ff_nn[i]] += wt_pre_trained_ff_nn
  class_counts[y_pred_rnn[i]] += wt_rnn
  sorted_dict = dict(sorted(class_counts.items(), key=lambda item: item[1], reverse=True))
  sorted_classes = list(sorted_dict.keys())
  most_frequent = sorted_classes[0]
  weight_vote_predictions.append(most_frequent)

In [29]:
# Weighted Vote Results
print('Weighted Ensemble BBC News Classification Accuracy is')
print(metrics.accuracy_score(y_true, weight_vote_predictions)*100)
print(classification_report(y_true, weight_vote_predictions, target_names = list(le_name_mapping.keys())))

Weighted Ensemble BBC News Classification Accuracy is
69.45169712793734
               precision    recall  f1-score   support

     business       0.84      0.62      0.72        79
entertainment       0.66      0.65      0.66        69
     politics       0.63      0.86      0.73        93
        sport       0.72      0.70      0.71        47
         tech       0.69      0.62      0.66        95

     accuracy                           0.69       383
    macro avg       0.71      0.69      0.69       383
 weighted avg       0.71      0.69      0.69       383



In [30]:
# Converting to CSV 

mv_df = pd.DataFrame()
mv_df['predictions'] = max_vote_predictions
mv_df.to_csv('max_vote.csv')

wt_df = pd.DataFrame()
wt_df['predictions'] = weight_vote_predictions
wt_df.to_csv('weight_vote.csv')

In [31]:
# Reading data again
init_df = pd.read_csv('initial_ff_nn.csv')
init_preds = init_df['predictions']
pre_train_df = pd.read_csv('pre_trained_ff_nn.csv')
pre_train_preds = pre_train_df['predictions']
rnn_df = pd.read_csv('rnn.csv')
rnn_preds = rnn_df['predictions']
mv_df = pd.read_csv('max_vote.csv')
mv_preds = mv_df['predictions']
wt_df = pd.read_csv('weight_vote.csv')
wt_preds = wt_df['predictions']

In [32]:
# Getting Performance Statistics
mv_ens_correct_1 = 0
wt_ens_correct_1 = 0

mv_ens_correct_2 = 0
wt_ens_correct_2 = 0

y_true = [idx for idx in np.argmax(test_Y, axis=1)]
for i in range(len(mv_preds)):
  if (init_preds[i] != y_true[i]) or (pre_train_preds[i] != y_true[i]) or (rnn_preds[i] != y_true[i]):
    if mv_preds[i] == y_true[i]:
      mv_ens_correct_1 += 1
    if wt_preds[i] == y_true[i]:
      wt_ens_correct_1 += 1
  if (init_preds[i] != y_true[i]) and (pre_train_preds[i] != y_true[i]) and (rnn_preds[i] != y_true[i]):
    if mv_preds[i] == y_true[i]:
      mv_ens_correct_2 += 1
    if wt_preds[i] == y_true[i]:
      wt_ens_correct_2 += 1

In [33]:
print('Number of instances wrongly classified by a base model but correctly classified by the ensemble')
print(mv_ens_correct_1)
print(wt_ens_correct_1)

print('Number of instances wrongly classified by all base models but correctly classified by the ensemble')
print(mv_ens_correct_2)
print(wt_ens_correct_2)

Number of instances wrongly classified by a base model but correctly classified by the ensemble
39
39
Number of instances wrongly classified by all base models but correctly classified by the ensemble
0
0
